In [1]:
import bionumpy as bnp
from bionumpy.bnpdataclass import BNPDataClass
import numpy as np
import re
from tempfile import NamedTemporaryFile
from fastatools import FastaFile
from contextlib import contextmanager

from typing import TypeVar
T = TypeVar("T", bound=BNPDataClass)

In [2]:
# package versions
%load_ext watermark
%watermark --iversions

numpy   : 1.23.5
re      : 2.2.1
bionumpy: 1.0.8



`fastatools` is a simple FASTA-parsing library I wrote that you can install with `pip install pyfastatools`

In [3]:
# bionumpy only works with ACGT characters
@contextmanager
def remove_ambiguous_chars(file: str):
    pattern = re.compile(r"N|W|K|R|S|B|D|H|Y|M|V")
    try:
        tmpfile = NamedTemporaryFile("w+", dir=".", suffix=".fna")
        for record in FastaFile(file).parse():
            record.sequence = pattern.sub("", record.sequence)
            record.write(tmpfile) # type: ignore
            
        tmpfile.seek(0)

        yield FastaFile(tmpfile.name)        
    finally:
        tmpfile.close()

In [4]:
@bnp.streamable(np.vstack)
def kmers(chunk: T, k: int = 4):
    seqs = bnp.change_encoding(chunk.sequence, bnp.DNAEncoding)
    counts = bnp.count_encoded(bnp.get_kmers(seqs, k=k))
    return counts.counts / counts.counts.sum(-1, keepdims=True)

This is the main function to use, taking as input a genome FASTA file

In [5]:
# results come back in the order of the input files
def process(file: str, is_vmag: bool = False) -> np.ndarray:
    with remove_ambiguous_chars(file) as fasta:
        stream = (
            bnp.open(
                fasta._file,
                buffer_type=bnp.io.MultiLineFastaBuffer()
            )
            .read_chunks()
        )

        kmer_freq = kmers(stream, k=4).astype(np.float32)

        # if input file is for a single vMAG, then all sequences are fragments of the same genome, so we should average them
        if is_vmag:
            kmer_freq = kmer_freq.mean(0, keepdims=True)

    return kmer_freq

The results from the above function can be stored in a variety of ways, but the most common would be `.h5` files (using the Python `hdf5` or `pytables` libraries).

The benefit of `.h5` files is that the outputs from all genomes can be stacked into a single matrix for storage. You can create a separate list to keep track of the order of genomes. 